<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/JSON_in_MySQL_Local_Shell_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

https://www.mysqltutorial.org/mysql-json/ <br>
https://www.databasestar.com/mysql-json/

#Install & Test Connectivity

In [ ]:
!apt-cache policy mysql-server

mysql-server:
  Installed: 8.0.32-0ubuntu0.20.04.2
  Candidate: 8.0.32-0ubuntu0.20.04.2
  Version table:
 *** 8.0.32-0ubuntu0.20.04.2 500
        500 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages
        500 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages
        100 /var/lib/dpkg/status
     8.0.19-0ubuntu5 500
        500 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages


In [ ]:
!apt-get update > /dev/null
!apt-get -y install mysql-server > /dev/null
!mysql --version
!/etc/init.d/mysql restart

mysql  Ver 8.0.32-0ubuntu0.20.04.2 for Linux on x86_64 ((Ubuntu))
 * Stopping MySQL database server mysqld
   ...done.
 * Starting MySQL database server mysqld
su: warning: cannot change directory to /nonexistent: No such file or directory
   ...done.


In [ ]:
!mysql -e 'create database praxisDB'
!mysql -e 'show databases'

+--------------------+
| Database           |
+--------------------+
| information_schema |
| mysql              |
| performance_schema |
| praxisDB           |
| sys                |
+--------------------+


#Create Table & Load Data

### Test Data <br>



In [ ]:
!mysql praxisDB -e 'drop table if exists events'
!mysql praxisDB -e '\
CREATE TABLE if not exists events( \
  id int auto_increment primary key, \
  event_name varchar(255), \
  visitor varchar(255), \
  properties json, \
  browser json \
);\
'
!mysql praxisDB -e 'desc events'

+------------+--------------+------+-----+---------+----------------+
| Field      | Type         | Null | Key | Default | Extra          |
+------------+--------------+------+-----+---------+----------------+
| id         | int          | NO   | PRI | NULL    | auto_increment |
| event_name | varchar(255) | YES  |     | NULL    |                |
| visitor    | varchar(255) | YES  |     | NULL    |                |
| properties | json         | YES  |     | NULL    |                |
| browser    | json         | YES  |     | NULL    |                |
+------------+--------------+------+-----+---------+----------------+


In [ ]:
%%writefile insert.sql
INSERT INTO events(event_name, visitor,properties, browser) 
VALUES 
('pageview', 
   '1',
   '{ "page": "/" }',
   '{ "name": "Safari", "os": "Mac", "resolution": { "x": 1920, "y": 1080 } }'
),
('pageview', 
  '2',
  '{ "page": "/contact" }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 2560, "y": 1600 } }'
),
(
  'pageview', 
  '1',
  '{ "page": "/products" }',
  '{ "name": "Safari", "os": "Mac", "resolution": { "x": 1920, "y": 1080 } }'
),
(
  'purchase', 
   '3',
  '{ "amount": 200 }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 1600, "y": 900 } }'
),
(
  'purchase', 
   '4',
  '{ "amount": 150 }',
  '{ "name": "Firefox", "os": "Windows", "resolution": { "x": 1280, "y": 800 } }'
),
(
  'purchase', 
  '4',
  '{ "amount": 500 }',
  '{ "name": "Chrome", "os": "Windows", "resolution": { "x": 1680, "y": 1050 } }'
);


Writing insert.sql


In [ ]:
!mysql praxisDB < insert.sql

In [ ]:
!mysql praxisDB -e 'select * from events'

+----+------------+---------+-----------------------+----------------------------------------------------------------------------+
| id | event_name | visitor | properties            | browser                                                                    |
+----+------------+---------+-----------------------+----------------------------------------------------------------------------+
|  1 | pageview   | 1       | {"page": "/"}         | {"os": "Mac", "name": "Safari", "resolution": {"x": 1920, "y": 1080}}      |
|  2 | pageview   | 2       | {"page": "/contact"}  | {"os": "Windows", "name": "Firefox", "resolution": {"x": 2560, "y": 1600}} |
|  3 | pageview   | 1       | {"page": "/products"} | {"os": "Mac", "name": "Safari", "resolution": {"x": 1920, "y": 1080}}      |
|  4 | purchase   | 3       | {"amount": 200}       | {"os": "Windows", "name": "Firefox", "resolution": {"x": 1600, "y": 900}}  |
|  5 | purchase   | 4       | {"amount": 150}       | {"os": "Windows", "name": "Fi

#Retrieve Data

In [ ]:
!mysql praxisDB -e "SELECT id, browser->'$.name' browser FROM events;"

+----+-----------+
| id | browser   |
+----+-----------+
|  1 | "Safari"  |
|  2 | "Firefox" |
|  3 | "Safari"  |
|  4 | "Firefox" |
|  5 | "Firefox" |
|  6 | "Chrome"  |
+----+-----------+


In [ ]:
# can you spot what is the difference with the previous query?
#
!mysql praxisDB -e "SELECT id, browser->>'$.name' browser FROM events;"

+----+---------+
| id | browser |
+----+---------+
|  1 | Safari  |
|  2 | Firefox |
|  3 | Safari  |
|  4 | Firefox |
|  5 | Firefox |
|  6 | Chrome  |
+----+---------+


In [ ]:
!mysql praxisDB -e "SELECT id, visitor, event_name, browser->>'$.name' browser FROM events where browser->'$.name'= 'Firefox';"

+----+---------+------------+---------+
| id | visitor | event_name | browser |
+----+---------+------------+---------+
|  2 | 2       | pageview   | Firefox |
|  4 | 3       | purchase   | Firefox |
|  5 | 4       | purchase   | Firefox |
+----+---------+------------+---------+


In [ ]:
!mysql praxisDB -e "SELECT id, visitor, event_name, \
                      browser->>'$.name' browser, \
                      browser->>'$.resolution' resolution \
                      FROM events \
                      where browser->'$.resolution.x'= 1920;\
"

+----+---------+------------+---------+------------------------+
| id | visitor | event_name | browser | resolution             |
+----+---------+------------+---------+------------------------+
|  1 | 1       | pageview   | Safari  | {"x": 1920, "y": 1080} |
|  3 | 1       | pageview   | Safari  | {"x": 1920, "y": 1080} |
+----+---------+------------+---------+------------------------+


In [ ]:
!mysql praxisDB -e "\
SELECT browser->>'$.name' browser, \
      count(browser) \
FROM events \
GROUP BY browser->>'$.name';\
"


+---------+----------------+
| browser | count(browser) |
+---------+----------------+
| Safari  |              2 |
| Firefox |              3 |
| Chrome  |              1 |
+---------+----------------+


In [ ]:
!mysql praxisDB -e "\
SELECT visitor, SUM(properties->>'$.amount') revenue \
FROM events \
WHERE properties->>'$.amount' > 0 \
GROUP BY visitor; \
"

+---------+---------+
| visitor | revenue |
+---------+---------+
| 3       |     200 |
| 4       |     650 |
+---------+---------+


#Python with Pandas

## Panda Modules

In [ ]:
!apt install libmysqlclient-dev > /dev/null
!pip install mysqlclient > /dev/null
import pandas as pd
import MySQLdb

In [ ]:
DBConn = MySQLdb.connect(db='praxisDB')
#df_mysql = pd.read_sql('select * from emp2;', con=con_mysql)  
df_mysql = pd.read_sql('show tables', con=DBConn)    
print ('loaded dataframe from MySQL. records:', len(df_mysql))
DBConn.close()

loaded dataframe from MySQL. records: 1


In [ ]:
df_mysql

,Tables_in_praxisDB
0,events


## Pandas Functions

In [ ]:
#To run any non-SELECT SQL command
def runCMD (DDL):
    DBConn= MySQLdb.connect(db='praxisDB')
    myCursor = DBConn.cursor()
    try:
        retcode = myCursor.execute(DDL)
        DBConn.commit()
    except :
        print('Error : return code = ',retcode)
    DBConn.close()

#To run any SELECT SQL command
def runSELECT (CMD):
    DBConn= MySQLdb.connect(db='praxisDB')
    df_mysql = pd.read_sql(CMD, con=DBConn)    
    DBConn.close()
    return df_mysql

#Operations with Python

In [ ]:
sql = "\
SELECT browser->>'$.name' browser, \
      count(browser) \
FROM events \
GROUP BY browser->>'$.name';\
"
runSELECT(sql)

,browser,count(browser)
0,Safari,2
1,Firefox,3
2,Chrome,1


#Chronobooks <br>
Two science fiction novels by Prithwis Mukerjee. A dystopian Earth. A technocratic society managed by artificial intelligence. Escape and epiphany on Mars. Can man and machine, carbon and silicon explore and escape into other dimensions of existence? An Indic perspective rooted in Advaita Vedanta and the Divine Feminine.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/CTCYFooter-1.png)